<a href="https://colab.research.google.com/github/lee813/Protein/blob/master/UCL_Pytorch_Regeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load dataset from Google cloud

In [1]:
from google.colab import drive
import numpy as np
import pandas as pd

drive.mount('/content/drive')




Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# =============================================================================
# Imports
# =============================================================================

import torch
import torch.nn.functional as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

import numpy as np
import argparse
import os
        
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split as tts


# =============================================================================
# Switches for what you want the model to do
# =============================================================================
cuda=True       # for training with gpu, make it true. For inference with cpu make false
load=False        # load in the model (default provided is 16 dimensional for nostruc data)
train=True      # Make true to train the model presuming you have the dataset
new_metal=True   # Make true to produce 'batch_size' samples of a given protein
                     # see the docs on github for description of how to do this

# trypsins = np.load('/content/drive/My Drive/Protein Data/trypsin140.npy', allow_pickle = True)
# print(trypsins.shape)
serine = np.load('/content/drive/My Drive/Protein Data/3.4.21_200.npy', allow_pickle = True)


In [3]:
print(serine.shape)

(41386, 4400)


# Model specification

 

In [4]:
args_dict = {
    "batch_size": 256,
    "num_epochs": 1000,
    "latent_dim": 16,
    "device": 0,
    "dataset": "nostruc",
    "lr": 5e-4
}


X_dim=4402
protein_dim = 4400

if cuda:
    # if args_dict["dataset"]=="nostruc":    
    #     data = np.load('/content/drive/My Drive/Protein Data/assembled_data_mb.npy')
       
    # else:
    #     data = []
    t3 = np.zeros((41386, 2))
    t4 = np.concatenate((serine, t3), axis = 1)
    data = t4

    print(data.shape)

    data, data_test=tts(data, test_size=0.15, shuffle=True)

    
    # np.concatenate((data, t4), axis = 0)
    n=data.shape[0]
else:
    print("No DATA")
    if args_dict["dataset"]=="nostruc":
        X_dim=4402
    else:
        X_dim=4353


if cuda:
    os.environ["CUDA_VISIBLE_DEVICES"]=str(args_dict['device'])

#spec batch size
batch_size=args_dict['batch_size']
#learning rate
lr=args_dict['lr']
# layer sizes
hidden_size=[512,256,128,args_dict['latent_dim']]

(41386, 4402)


In [0]:
data_test[4][0:20]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

#Model

In [0]:
# =============================================================================
# Module
# =============================================================================
class feed_forward(torch.nn.Module):
    def __init__(self, input_size, hidden_sizes, batch_size):
        super().__init__()
        
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.batch_size = batch_size
           

        self.fc = torch.nn.Linear(input_size, hidden_sizes[0])  # 2 for bidirection 
        self.BN = torch.nn.BatchNorm1d(hidden_sizes[0])
        self.fc1 = torch.nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.BN1 = torch.nn.BatchNorm1d(hidden_sizes[1])
        self.fc2 = torch.nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.BN2 = torch.nn.BatchNorm1d(hidden_sizes[2])
        self.fc3_mu = torch.nn.Linear(hidden_sizes[2], hidden_sizes[3])
        self.fc3_sig = torch.nn.Linear(hidden_sizes[2], hidden_sizes[3])
        
        if args_dict["dataset"]=="struc":
            self.fc4 = torch.nn.Linear(hidden_sizes[3]+1273, hidden_sizes[2])
        else:        
            self.fc4 = torch.nn.Linear(hidden_sizes[3]+2, hidden_sizes[2])
        self.BN4 = torch.nn.BatchNorm1d(hidden_sizes[2])
        self.fc5 = torch.nn.Linear(hidden_sizes[2], hidden_sizes[1])
        self.BN5 = torch.nn.BatchNorm1d(hidden_sizes[1])
        self.fc6 = torch.nn.Linear(hidden_sizes[1], hidden_sizes[0])
        self.BN6 = torch.nn.BatchNorm1d(hidden_sizes[0])
        if args_dict["dataset"]=="struc":
            self.fc7 = torch.nn.Linear(hidden_sizes[0], input_size-1273)
        else:
            self.fc7 = torch.nn.Linear(hidden_sizes[0], input_size-2)
            
    def sample_z(self, mu, log_var):
        # Using reparameterization trick to sample from a gaussian
        
        if cuda:
            eps = torch.randn(self.batch_size, self.hidden_sizes[-1]).cuda()
        else:
            eps = torch.randn(self.batch_size, self.hidden_sizes[-1])
	
        return mu + torch.exp(log_var / 2) * eps
    
    def forward(self, x, code, struc=None):
        
        ###########
        # Encoder #
        ###########
        
        # get the code from the tensor
        # add the conditioned code
        if args_dict["dataset"]!="struc":
            x = torch.cat((x,code),1)
        else:
            x = torch.cat((x,code,struc),1)        
        # Layer 0
        out1 = self.fc(x)        
        out1 = nn.relu(self.BN(out1))
        # Layer 1
        out2 = self.fc1(out1)
        out2 = nn.relu(self.BN1(out2))
        # Layer 2
        out3 = self.fc2(out2)
        out3 = nn.relu(self.BN2(out3))
        # Layer 3 - mu
        mu   = self.fc3_mu(out3)
        # layer 3 - sig
        sig  = nn.softplus(self.fc3_sig(out3))    

         ###########
        # Decoder #
        ###########
        
        # sample from the distro
        sample= self.sample_z(mu, sig)
        # add the conditioned code
        if args_dict["dataset"]!="struc": 
            sample = torch.cat((sample, code),1)
        else:
            sample = torch.cat((sample, code, struc),1)
        # Layer 4
        out4 = self.fc4(sample)
        out4 = nn.relu(self.BN4(out4))
        # Layer 5
        out5 = self.fc5(out4)
        out5 = nn.relu(self.BN5(out5))
        # Layer 6
        out6 = self.fc6(out5)
        out6 = nn.relu(self.BN6(out6))
        # Layer 7
        out7 = nn.sigmoid(self.fc7(out6))
        
        return out7, mu, sig


#Train



In [0]:
# =============================================================================
# Training
# =============================================================================

# init the networks
if cuda:
    ff = feed_forward(X_dim, hidden_size, batch_size).cuda()
else:
    ff = feed_forward(X_dim, hidden_size, batch_size)

# change the loading bit here
if load:
    ff.load_state_dict(torch.load("models/metal16_nostruc", map_location=lambda storage, loc: storage))


# Loss and Optimizer
solver = optim.Adam(ff.parameters(), lr=lr)
burn_in_counter = 0
tick = 0


# number of epochs
num_epochs=args_dict['num_epochs']
# number of epochs
num_epochs=args_dict['num_epochs']

if train:
    for its in range(num_epochs):
        
        #############################
        # TRAINING 
        #############################
        
        ff.train()
        scores=[]
        data=shuffle(data)
        print("Grammar Cond. - Epoch: {0}/{1}  Latent: {2}".format(its,num_epochs,hidden_size[-1]))
        for it in range(n // batch_size):
        
            if args_dict["dataset"]=="nostruc":
                
                x_batch=data[it * batch_size: (it + 1) * batch_size]
                code = x_batch[:,-2:]
                x_batch = x_batch[:,:protein_dim]

                if cuda:
                    X = torch.from_numpy(x_batch).cuda().type(torch.cuda.FloatTensor)
                    C = torch.from_numpy(code).cuda().type(torch.cuda.FloatTensor)
                else:
                    X = torch.from_numpy(x_batch).type(torch.FloatTensor)
                    C = torch.from_numpy(code).type(torch.FloatTensor)

                
            else:
                x_batch=data[it * batch_size: (it + 1) * batch_size]
                code = x_batch[:,-2:]
                structure = x_batch[:,protein_dim:-2]
                x_batch = x_batch[:,:protein_dim]

                if cuda:
                    X = torch.from_numpy(x_batch).cuda().type(torch.cuda.FloatTensor)
                    C = torch.from_numpy(code).cuda().type(torch.cuda.FloatTensor)
                    S = torch.from_numpy(structure).cuda().type(torch.cuda.FloatTensor) 
                else:
                    X = torch.from_numpy(x_batch).type(torch.FloatTensor)
                    C = torch.from_numpy(code).type(torch.FloatTensor)
                    S = torch.from_numpy(structure).type(torch.FloatTensor)  
    

            
            #turf last gradients
            solver.zero_grad()
            
            
            if args_dict["dataset"]=="struc":
            # Forward
                x_sample, z_mu, z_var = ff(X, C, S)
            else:
                x_sample, z_mu, z_var = ff(X, C)
            
    
                
            # Loss
            recon_loss = nn.binary_cross_entropy(x_sample, X, size_average=False) # by setting to false it sums instead of avg.
            kl_loss = 0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1. - z_var)
            #kl_loss=KL_Div(z_mu,z_var,unit_gauss=True,cuda=True)
            kl_loss = kl_loss*burn_in_counter
            loss = recon_loss + kl_loss
            
            
            # Backward
            loss.backward()
        
            # Update
            solver.step()
            
            
            
            len_aa=protein_dim
            y_label=np.argmax(x_batch[:,:len_aa].reshape(batch_size,-1,22), axis=2)
            y_pred =np.argmax(x_sample[:,:len_aa].cpu().data.numpy().reshape(batch_size,-1,22), axis=2)
            

            
            # can use argmax again for clipping as it uses the first instance of 21
            # loop with 256 examples is only about 3 milliseconds                      
            for idx, row in enumerate(y_label):
                scores.append(accuracy_score(row[:np.argmax(row)],y_pred[idx][:np.argmax(row)]))
        
        print("Tra Acc: {0}".format(np.mean(scores)))
                
        if its==(num_epochs-1):
            with open('latent_results_'+str(args_dict["dataset"])+'.txt', 'a') as f:
                f.write(str(args_dict['latent_dim'])+' train '+str(np.mean(scores)))


        
        if its>400 and burn_in_counter<1.0:
            burn_in_counter+=0.003
        
        
        
        #############################
        # Validation 
        #############################
        
        scores=[]
        
        ff.eval()
        for it in range(data_test.shape[0] // batch_size):
            x_batch=data_test[it * batch_size: (it + 1) * batch_size]

            if args_dict["dataset"]=="nostruc":

                x_batch=data[it * batch_size: (it + 1) * batch_size]
                code = x_batch[:,-2:]
                x_batch = x_batch[:,:protein_dim]

                if cuda:
                    X = torch.from_numpy(x_batch).cuda().type(torch.cuda.FloatTensor)
                    C = torch.from_numpy(code).cuda().type(torch.cuda.FloatTensor)
                else:
                    X = torch.from_numpy(x_batch).type(torch.FloatTensor)
                    C = torch.from_numpy(code).type(torch.FloatTensor)


            else:
                
                x_batch=data[it * batch_size: (it + 1) * batch_size]
                code = x_batch[:,-2:]
                structure = x_batch[:,protein_dim:-2]
                x_batch = x_batch[:,:protein_dim]

                if cuda:
                    X = torch.from_numpy(x_batch).cuda().type(torch.cuda.FloatTensor)
                    C = torch.from_numpy(code).cuda().type(torch.cuda.FloatTensor)
                    S = torch.from_numpy(structure).cuda().type(torch.cuda.FloatTensor)
                else:
                    X = torch.from_numpy(x_batch).type(torch.FloatTensor)
                    C = torch.from_numpy(code).type(torch.FloatTensor)
                    S = torch.from_numpy(structure).type(torch.FloatTensor)


            if args_dict["dataset"]=="struc":
            # Forward
                x_sample, z_mu, z_var = ff(X, C, S)
            else:
                x_sample, z_mu, z_var = ff(X, C)

            # Validation Loss
            recon_loss = nn.binary_cross_entropy(x_sample, X, size_average=False) # by setting to false it sums instead of avg.
            kl_loss = 0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1. - z_var)
            #kl_loss=KL_Div(z_mu,z_var,unit_gauss=True,cuda=True)
            kl_loss = kl_loss*burn_in_counter
            loss = recon_loss + kl_loss
            print(f"Validation total loss {loss}")

        
            len_aa=protein_dim
            y_label=np.argmax(x_batch[:,:len_aa].reshape(batch_size,-1,22), axis=2)
            y_pred =np.argmax(x_sample[:,:len_aa].cpu().data.numpy().reshape(batch_size,-1,22), axis=2)

            for idx, row in enumerate(y_label):
                l = row[:np.argmax(row)]
                p = y_pred[idx][:np.argmax(row)]
                acc = accuracy_score(row[:np.argmax(row)],y_pred[idx][:np.argmax(row)])
                scores.append(acc)


        print("Val Acc: {0}".format(np.mean(scores)))

        if its==(num_epochs-1):
            with open('latent_results_'+str(args_dict["dataset"])+'.txt', 'a') as f:
                f.write(str(args_dict['latent_dim'])+' test '+str(np.mean(scores)))



# saves if its running on gpu          
import time
if cuda:
    torch.save(ff.state_dict(), '/content/drive/My Drive/model/metal'+str(args_dict['latent_dim'])+"_"
    +str(args_dict['dataset'] + str(time.time())))


Grammar Cond. - Epoch: 0/1400  Latent: 16


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Tra Acc: 0.1497058922389787
Validation total loss 156960.84375
Validation total loss 159099.6875
Validation total loss 157396.0625
Validation total loss 147745.75
Validation total loss 158539.59375
Validation total loss 155684.109375
Validation total loss 156140.34375
Validation total loss 155462.84375
Validation total loss 158825.203125
Validation total loss 158937.1875
Validation total loss 156228.1875
Validation total loss 157452.203125
Validation total loss 158791.4375
Validation total loss 157191.140625
Validation total loss 157919.265625
Validation total loss 156324.15625
Validation total loss 156425.78125
Validation total loss 158370.734375
Validation total loss 155334.796875
Validation total loss 155786.984375
Validation total loss 155865.21875
Validation total loss 157049.09375
Validation total loss 158956.84375
Validation total loss 152851.25
Val Acc: 0.21269260779158636
Grammar Cond. - Epoch: 1/1400  Latent: 16
Tra Acc: 0.23584298802708348
Validation total loss 143946.3125
V

# Launch model


In [0]:
def newMetalBinder(model,data,name):
    """
    Generates a new sequence based on a metal code and a grammar. 
    The data array is (4353,) where the first 3080 are the
    sequence, the next 1265 are the fold and the final 8 are the metal
    binding flags. Fold is optional
    """
    scores=[]
    #model into eval mode    
    model.eval()
    
    if args_dict["dataset"]=="nostruc":

        code = np.tile(data[-8:],(model.batch_size,1))
        x = np.tile(data[:protein_dim],(model.batch_size,1))

        if cuda:
            X = torch.from_numpy(x).cuda().type(torch.cuda.FloatTensor)
            C = torch.from_numpy(code).cuda().type(torch.cuda.FloatTensor)
        else:
            X = torch.from_numpy(x).type(torch.FloatTensor)
            C = torch.from_numpy(code).type(torch.FloatTensor)


    else:
        
        code = np.tile(data[-8:],(model.batch_size,1))
        structure = np.tile(data[protein_dim:-8],(model.batch_size,1))
        x = np.tile(data[:protein_dim],(model.batch_size,1))

        if cuda:
            X = torch.from_numpy(x).cuda().type(torch.cuda.FloatTensor)
            C = torch.from_numpy(code).cuda().type(torch.cuda.FloatTensor)
            S = torch.from_numpy(structure).cuda().type(torch.cuda.FloatTensor)
        else:
            X = torch.from_numpy(x).type(torch.FloatTensor)
            C = torch.from_numpy(code).type(torch.FloatTensor)
            S = torch.from_numpy(structure).type(torch.FloatTensor)


    if args_dict["dataset"]=="struc":
    # Forward
        x_sample, z_mu, z_var = ff(X, C, S)
    else:
        x_sample, z_mu, z_var = ff(X, C)
    
    
    len_aa=140*22
    y_label=np.argmax(x[:,:len_aa].reshape(batch_size,-1,22), axis=2)
    y_pred =np.argmax(x_sample[:,:len_aa].cpu().data.numpy().reshape(batch_size,-1,22), axis=2)
    #np.save(name,y_pred)
    print(x_sample[:,:len_aa].cpu().data.numpy().reshape(batch_size,-1,22).shape)
    np.save(name,x_sample[:,:len_aa].cpu().data.numpy().reshape(batch_size,-1,22))
    
    
    for idx, row in enumerate(y_label):
        scores.append(accuracy_score(row[:np.argmax(row)],y_pred[idx][:np.argmax(row)]))
    print("Accuracy: {0}".format(np.mean(scores)))
    
    return